In [84]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint
import csv
from copy import deepcopy
import json
import pathlib
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import src.annotation_utils as a_utils
import src.llm_utils as llm_utils
import os

load_dotenv()

BRAT_DATA_PATH = os.environ['BRAT_DATA_PATH']
DATA_DEF_FILE = os.environ['DATA_CATEGORY_DEFINITION']


## Load brat dataset

In [27]:

import dataclasses

from pybrat.parser import BratParser, Entity, Event, Example, Relation

# Initialize a parser.
brat = BratParser(error="ignore")
# brat = BratParser()
annotations = brat.parse(BRAT_DATA_PATH)


In [37]:
'''
Test the loaded annotation
'''

[e for x in annotations for e in x.entities]
# flag = False
for x in annotations:
    # for e in [n for l in [x.entities, x.relations, x.events] for n in l]:
    #     if e.references:
    #         print(e)
    # # print(x.text)
    pprint(x.entities)
    # pprint(x.relations)
    # print(x.events)
    # if flag: break
    # for e in x.events:
    #     # pprint(e)
    #     # if flag: break
    #     for e2 in x.events:
    #         if e == e2: continue
    #         if e.trigger.spans[0] == e2.trigger.spans[0]:
    #             # pprint(x.text)
    #             pprint([e.type, e2.type])
    #             # flag = True
    #             # break
# assert all(isinstance(e, Relation) for x in annotations for e in x.relations)
# assert all(isinstance(e, Event) for x in annotations for e in x.events)

[]
[]
[Entity(mention='you visit or access our Services',
        type='Condition',
        spans=[Span(start=40, end=72)],
        references=[],
        id='T1'),
 Entity(mention='you',
        type='User',
        spans=[Span(start=40, end=43)],
        references=[],
        id='T2'),
 Entity(mention='our',
        type='First-party-entity',
        spans=[Span(start=60, end=63)],
        references=[],
        id='T3'),
 Entity(mention='use',
        type='collection-use',
        spans=[Span(start=119, end=122)],
        references=[],
        id='T4'),
 Entity(mention='automatically collect information',
        type='RecordManagement',
        spans=[Span(start=292, end=325)],
        references=[],
        id='T14'),
 Entity(mention='online behavior',
        type='BrowsingBehavior',
        spans=[Span(start=345, end=360)],
        references=[],
        id='T16'),
 Entity(mention='device',
        type='DeviceBased',
        spans=[Span(start=378, end=384)],
        referenc

In [72]:
# data_entities = a_utils.get_data_entities_of_segments(annotations, DATA_DEF_FILE)
data_entities = a_utils.get_data_entities_of_sentences(annotations, DATA_DEF_FILE)

len(data_entities), data_entities[10]

(1067,
 {'segment': 'INTRODUCTION\n\nDomains Holdings Group Limited (the "Company"), offers a global network of content-rich web, mobile and social sites, including user acquisition and media buying (including paid search, display, mobile and social advertising), content optimization (including key word search, off-page and on-page optimization), and affiliate program, both in the web and mobile ecosystem (including through websites or mobile apps operated by Company; collectively, the "Services").\nThe Company is deeply committed to safeguard the privacy expectations of its end users ("User(s)", "you" or "your"). Accordingly, we have put in place this Privacy Policy, which outlines our data protection practices, including how we collect, use, disclose and protect your Personal Information, as well as your rights with respect to your Personal Information.\nIn this Privacy Policy, you can learn about:\nWHAT INFORMATION WE COLLECT?\nMINORS\nHOW WE USE THE COLLECTED INFORMATION?\nMARKETIN

## Select training data

In [73]:
def has_no_entities(segment):
    return not segment["entities"]

empty_entity_indices = [i for i, segment in enumerate(data_entities) if has_no_entities(segment)]

training_set_indices = []
validation_set_indices = []
for i, segment in enumerate(data_entities):
    if not has_no_entities(segment):
        if len(training_set_indices) < 20:
            training_set_indices.append(i)
        elif len(validation_set_indices) < 3:
            validation_set_indices.append(i)

for i in empty_entity_indices:
    if len(training_set_indices) < 30:
        training_set_indices.append(i)
    elif len(validation_set_indices) < 5:
        validation_set_indices.append(i)

len(training_set_indices), len(validation_set_indices)

(30, 5)

## Fine-tune model with specified training data

In [76]:
# all_data = llm_utils.as_training_data_for_data_span_of_segment(data_entities)
all_data = llm_utils.as_training_data_for_data_span_of_sentence(data_entities)
base_model = 'gpt-4o-mini-2024-07-18'
suffix = 'seg-sent-data-entities-30_train-5_val-with_empty'

In [78]:
job_desc_dir, fine_tune_job, test_set = llm_utils.fine_tune_with_data(
    all_data, training_set_indices, validation_set_indices,
    BRAT_DATA_PATH, DATA_DEF_FILE,
    basemodel=base_model,
    fine_tune_args={'suffix':suffix})

job_desc_dir, fine_tune_job, len(test_set), test_set[0]

Constructing training and validation data files (locally)...
Uploading training and validation data files...
Data files uploading (API calls returned).
Creating fine-tuning job...


InternalServerError: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_1ba4e940c152a7b11114adae3f298f86 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}

In [41]:
llm_utils.await_fine_tune_finish_and_clean_up(wait_for_job_completion=False)

FineTuningJob(id='ftjob-61c0FVgYkOfjZaMwRpOjNOzo', created_at=1726265855, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:rui:30-train-5-val-with-empty-from-api:A790JXld', finished_at=1726266269, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-B2C2pNzAq4paAOvhIYdFJlSv', result_files=['file-1TTTIJ6t7DVXg51yYafkwd4J'], seed=1504396448, status='succeeded', trained_tokens=68019, training_file='file-iYtsamFpHaR3FyqGhHWNeMVm', validation_file='file-C1S6NC5e9VWZrwgmuFzkDxVR', estimated_finish=None, integrations=[], user_provided_suffix='30_train-5_val-with_empty-from_api')

In [31]:
fine_tuned_model_id = 'ft:gpt-4o-mini-2024-07-18:rui:30_train-5_val-content_only-from_api:A6KhgqGM'
fine_tuned_model_id = 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-content-only-from-api:A6cig7w6'

## Run model evaluation

In [63]:
training_set, validation_set, test_set, fine_tuned_model_id = llm_utils.load_eval_info()

len(training_set), len(validation_set), len(test_set)

In [64]:
messages_list = [data['messages'][:-1] for data in test_set]
correct_outputs = [data['messages'][-1]['content'] for data in test_set]
dir_name, obj_model_outputs = llm_utils.query_llm(fine_tuned_model_id, messages_list, correct_outputs=correct_outputs)

dir_name, len(obj_model_outputs)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]


('eval-2024-09-14-00-12-51-ft:gpt-4o-mini-2024-07-18:rui:30-training-data-with-content-and-10-val:A6KhgqGM',
 10)

### Or, evaluate an existing model
Thus we don't need to split training data out

In [19]:
custom_model_id = 'gpt-4o-mini-2024-07-18'
all_data = llm_utils.as_training_data_for_data_span(data_entities)
messages_list = [data['messages'][:-1] for data in all_data]
correct_outputs = [data['messages'][-1]['content'] for data in all_data]
dir_name, obj_model_outputs = llm_utils.query_llm(custom_model_id, messages_list, correct_outputs=correct_outputs)

dir_name, len(obj_model_outputs)

100%|██████████| 181/181 [03:05<00:00,  1.03s/it]


('eval-2024-09-13-22-59-04-gpt-4o-mini-2024-07-18', 181)

## Calculate statistics from model performance (from saved query results)

In [3]:
dir_name = '2024-09-13-00-44-00-6cbf289f-798c-46ff-8d72-221b0da1099e'
dir_name = '2024-09-13-14-48-28-ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-no-empty-from-api:A70TvfoL'

In [24]:
desc, saved_queries = list(llm_utils.load_saved_llm_queries())
desc, len(saved_queries), saved_queries[0]

({'model': 'gpt-4o-mini-2024-07-18'},
 181,
 {'input': [{'role': 'system',
    'content': 'You are an annotation expert. You will be given a segment of a privacy policy of a web or mobile application, and will be asked to annotate entities in it.'},
   {'role': 'user',
    'content': 'Please annotate the given segment of privacy policy for data entities, and adhere to the following guidelines:\n\nIMPORTANT: Filtering Out General Phrases\nBefore annotating, carefully check each potential data entity. DO NOT annotate general phrases that do not provide specific data types.\nExamples of general phrases to omit include, but are not limited to:\n\n"your personal information"\n"the information you share (with us)"\n"the information we collect about you"\n"other data"\n"information about you"\n"personal information"\n"user information"\n"customer data"\n"collected information"\n"any information"\n\nIf a phrase does not clearly indicate a specific type of personal data, DO NOT include it in yo

In [25]:
import numpy as np
import sys

# num_datapoints = len(saved_queries)

def calc_statistics(saved_queries):
    result_score_list = []
    empty_result_score_list = []
    non_empty_result_score_list = []

    failed = {}
    for i, (model_output, correct_output) in enumerate([(query['output'], query['correct_output']) for query in saved_queries]):
        try:
            model_output_parsed = json.loads(model_output)
        except json.JSONDecodeError as e:
            failed[i] = (model_output, correct_output)
            continue
        correct_output_parsed = json.loads(correct_output)
        try:
            result_score = llm_utils.precision_accuracy_f1(correct_output_parsed, model_output_parsed)
        except TypeError as e:
            failed[i] = (model_output, correct_output)
            continue
        result_score_list.append(result_score)
        if correct_output_parsed:
            non_empty_result_score_list.append(result_score)
        else:
            empty_result_score_list.append(result_score)

    return result_score_list, non_empty_result_score_list, empty_result_score_list, failed


def calc_and_print_statistics(desc, saved_queries):
    result_score_list, non_empty_result_score_list, empty_result_score_list, failed = calc_statistics(saved_queries)

    print(f"Stat for eval with desc: {desc}")
    print(f"  {len(result_score_list)} valid datapoints, avg. precission, recall, f1:", np.mean(result_score_list, axis=0))
    print(f"  {len(non_empty_result_score_list)} (ought to be) non-empty datapoints, avg. precission, recall, f1:", np.mean(non_empty_result_score_list, axis=0))
    print(f"  {len(empty_result_score_list)} (ought to be) empty datapoints, avg. precission, recall, f1:", np.mean(empty_result_score_list, axis=0))
    print(f"  {len(failed)} datapoints are not valid JSON")
    print("  ", end=''), pprint(failed)


calc_and_print_statistics(desc, saved_queries)

Stat for eval with desc: {'model': 'gpt-4o-mini-2024-07-18'}
  74 valid datapoints, avg. precission, recall, f1: [0.94594595 0.94594595 0.94594595]
  4 (ought to be) non-empty datapoints, avg. precission, recall, f1: [0. 0. 0.]
  70 (ought to be) empty datapoints, avg. precission, recall, f1: [1. 1. 1.]
  107 datapoints are not valid JSON
  {1: ('```json\n[]\n```', '[]'),
 3: ('```json\n[]\n```', '[]'),
 5: ('```json\n[]\n```', '[]'),
 6: ('```json\n[]\n```', '[]'),
 10: ('```json\n'
      '[\n'
      '    {\n'
      '        "context_type": "first-party-collection-use",\n'
      '        "data_entity": "full name"\n'
      '    },\n'
      '    {\n'
      '        "context_type": "first-party-collection-use",\n'
      '        "data_entity": "email address"\n'
      '    },\n'
      '    {\n'
      '        "context_type": "first-party-collection-use",\n'
      '        "data_entity": "device\'s unique identifiers"\n'
      '    },\n'
      '    {\n'
      '        "context_type": "fi

In [4]:
# For query '2024-09-13-00-44-00-6cbf289f-798c-46ff-8d72-221b0da1099e'
# For model 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-content-only-from-api:A6cig7w6'
# print(np.mean(result_score_list, axis=0))
print(f"{len(result_score_list)} valid datapoints, avg. precission, recall, f1:", np.mean(result_score_list, axis=0))
print(f"{len(non_empty_result_score_list)} (ought to be) non-empty datapoints, avg. precission, recall, f1:", np.mean(non_empty_result_score_list, axis=0))
print(f"{len(empty_result_score_list)} (ought to be) empty datapoints, avg. precission, recall, f1:", np.mean(empty_result_score_list, axis=0))
print(f"{len(failed)} datapoints are not valid JSON")
failed

146 valid datapoints, avg. precission, recall, f1: [0.02842466 0.02796804 0.02663079]
10 (ought to be) non-empty datapoints, avg. precission, recall, f1: [0.415      0.40833333 0.38880952]
136 (ought to be) empty datapoints, avg. precission, recall, f1: [0. 0. 0.]
0 datapoints are not valid JSON


{}

In [30]:
# For query '2024-09-13-11-25-49-a393b3ea-6ae0-4433-8e92-610bd63e1305'
# For model 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-with-empty-from-api:A6o1jAxy'

print(f"{len(result_score_list)} valid datapoints, avg. precission, recall, f1:", np.mean(result_score_list, axis=0))
print(f"{len(non_empty_result_score_list)} (ought to be) non-empty datapoints, avg. precission, recall, f1:", np.mean(non_empty_result_score_list, axis=0))
print(f"{len(empty_result_score_list)} (ought to be) empty datapoints, avg. precission, recall, f1:", np.mean(empty_result_score_list, axis=0))
print(f"{len(failed)} datapoints are not valid JSON")
failed

144 valid datapoints, avg. precission, recall, f1: [0.78908855 0.80578704 0.78842856]
21 (ought to be) non-empty datapoints, avg. precission, recall, f1: [0.2680358  0.38253968 0.26351015]
123 (ought to be) empty datapoints, avg. precission, recall, f1: [0.87804878 0.87804878 0.87804878]
2 datapoints are not valid JSON


{54: ('[] reflections of persona data entities []', '[]'),
 67: ('["name": "Device ID", "type": "identification numbers associated with your device", "type": "personal information"]',
  '["geographical location data", "similar information regarding the location of your mobile device", "location data", "web request", "browser type", "referring / exit pages and URLs", "domain names", "landing pages"]')}

In [24]:
def compare_two_evaluations(eval1, eval2):
    desc1, saved_queries1 = eval1
    desc2, saved_queries2 = eval2

    def to_reserve_map(saved_queries):
        reverse_map = {}
        for query in saved_queries:
            key = (str(query['input'][1]), str(query['correct_output']))
            reverse_map[key] = query
        return reverse_map

    reverse_map1 = to_reserve_map(saved_queries1)
    reverse_map2 = to_reserve_map(saved_queries2)

    ## Calculate what keys are common, and what keys are unique to each
    common_keys = set(reverse_map1.keys()) & set(reverse_map2.keys())
    unique_keys1 = set(reverse_map1.keys()) - set(reverse_map2.keys())
    unique_keys2 = set(reverse_map2.keys()) - set(reverse_map1.keys())

    print(len(common_keys), len(unique_keys1), len(unique_keys2))

    # Return the common and unique queries
    common_queries1 = []
    common_queries2 = []
    unique_queries1 = []
    unique_queries2 = []
    for key in common_keys:
        common_queries1.append(reverse_map1[key])
        common_queries2.append(reverse_map2[key])
    for key in unique_keys1:
        unique_queries1.append(reverse_map1[key])
    for key in unique_keys2:
        unique_queries2.append(reverse_map2[key])

    return common_queries1, common_queries2, unique_queries1, unique_queries2


queries_to_load = [
    # '2024-09-13-00-44-00-6cbf289f-798c-46ff-8d72-221b0da1099e',
    '2024-09-13-11-25-49-a393b3ea-6ae0-4433-8e92-610bd63e1305',
    '2024-09-13-14-48-28-ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-no-empty-from-api:A70TvfoL',
]
eval1, eval2 = [llm_utils.load_saved_llm_queries(query) for query in queries_to_load[:2]]
common_queries1, common_queries2, unique_queries1, unique_queries2 = compare_two_evaluations(eval1, eval2)


calc_and_print_statistics(eval1[0], common_queries1)
calc_and_print_statistics(eval2[0], common_queries2)
calc_and_print_statistics(eval1[0], unique_queries1)
calc_and_print_statistics(eval2[0], unique_queries2)

134 12 12
Stat for eval with desc: {'model': 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-with-empty-from-api:A6o1jAxy'}
  133 valid datapoints, avg. precission, recall, f1: [0.81938179 0.8245614  0.82119585]
  10 (ought to be) non-empty datapoints, avg. precission, recall, f1: [0.09777778 0.16666667 0.12190476]
  123 (ought to be) empty datapoints, avg. precission, recall, f1: [0.87804878 0.87804878 0.87804878]
  1 datapoints are not valid JSON
  {53: ('[] reflections of persona data entities []', '[]')}
Stat for eval with desc: {'model': 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-no-empty-from-api:A70TvfoL'}
  134 valid datapoints, avg. precission, recall, f1: [0.02860697 0.03358209 0.03059701]
  10 (ought to be) non-empty datapoints, avg. precission, recall, f1: [0.38333333 0.45       0.41      ]
  124 (ought to be) empty datapoints, avg. precission, recall, f1: [0. 0. 0.]
  0 datapoints are not valid JSON
  {}
Stat for eval with desc: {'model': 'ft:gpt-4o-mini-2024-07-18:rui:30

In [22]:
%debug

> /usr/lib/python3.12/json/decoder.py(340)decode()
    338         end = _w(s, end).end()
    339         if end != len(s):
--> 340             raise JSONDecodeError("Extra data", s, end)
    341         return obj
    342 

'[] reflections of persona data entities []'
'[] reflections of persona data entities []'
'[] reflections of persona data entities []'
'[] reflections of persona data entities []'


## Test code

In [62]:
# Test how function calls works for normal conversation, beforing turing that into training data.

from openai import OpenAI


assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="You are an legal annotation expert. Use you knowledge base to answer questions about privacy policies.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)


def upload_doc():
    vector_store = client.beta.vector_stores.get(name="Data Categories")

    # Ready the files for upload to OpenAI
    file_paths = [DATA_DEF_FILE]
    file_streams = [open(path, "rb") for path in file_paths]

    # Use the upload and poll SDK helper to upload the files, add them to the vector store,
    # and poll the status of the file batch for completion.
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
    )
    return file_batch


# file_path = upload_doc()

vector_store_id = 'vs_YqhT8wThRGO9ynsFlAipwVxD'

In [ ]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

In [ ]:
# Continuing test function call

client.

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}},
)

# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("edgar/aapl-10k.pdf", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": 'What should "" be classified as?',
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

In [ ]:

# The praser returns dataclasses.
assert len(examples) == 80
assert all(isinstance(x, Example) for x in examples)
assert all(isinstance(e, Entity) for x in examples for e in x.entities)
assert all(isinstance(e, Relation) for x in examples for e in x.relations)
assert all(isinstance(e, Event) for x in examples for e in x.events)

id_ = "BioNLP-ST_2011_EPI/PMID-19377285"
example = next(x for x in examples if x.id == id_)
print(example.text)
print(len(example.entities), next(iter(example.entities)))
print(len(example.relations), next(iter(example.relations)))
print(len(example.events), next(iter(example.events)))

# Use dataclasses.asdict to convert examples to dictionaries.
examples = [*map(dataclasses.asdict, examples)]
assert all(isinstance(x, dict) for x in examples)
assert all(isinstance(e, dict) for x in examples for e in x["entities"])
assert all(isinstance(e, dict) for x in examples for e in x["relations"])
assert all(isinstance(e, dict) for x in examples for e in x["events"])

print(examples[0])